In [ ]:
import random
import pickle
import mlflow


class GAlgorithm:

  def __init__(self, Config, dfPlayers):
    self.Config = Config
    self.dfPlayers=dfPlayers.copy()

  
  def initilization_of_population(self):
      population = []
      dfelements=self.dfPlayers
      SquadComposition=self.Config["SquadComposition"]
      for i in range(self.Config["PopSize"]):
          validTeam=0
          Niterations=self.Config["ErrorAttempts"]
          k=0
          #print(f"--->{Niterations} \n----> {SquadComposition}")
          while validTeam==0 and k<Niterations:
            chromosomeGK = dfelements[dfelements["position"]=="GK"].sample(SquadComposition["Goalkeepers"]).sort_values(by=['position', 'total_points'])
            chromosomeDEF = dfelements[dfelements["position"]=="DEF"].sample(SquadComposition["Defenders"]).sort_values(by=['position', 'total_points'])
            chromosomeMED = dfelements[dfelements["position"]=="MID"].sample(SquadComposition["Midfielders"]).sort_values(by=['position', 'total_points'])
            chromosomeFWD = dfelements[dfelements["position"]=="FWD"].sample(SquadComposition["Forwards"]).sort_values(by=['position', 'total_points'])
            
            #print(f"###> {k}: {chromosomeGK}, {chromosomeDEF},{chromosomeGK},{chromosomeGK} ")
            validTeam=1
            k+=1
            dfTMP=pd.concat([chromosomeGK, chromosomeDEF, chromosomeMED , chromosomeFWD])
            #Check Max elements per team
            if max(dfTMP.groupby(['team']).size()) > self.Config["SquadComposition"]["Sameteam"]:
              validTeam=0
            #Check max Cost
            if dfTMP["value"].sum() > self.Config["SquadComposition"]["MaxCost"]:
              validTeam=0
            # Check if team already exists
            for dfRef in population:
              if dfTMP.equals(dfRef):
                validTeam=0
                break
            if validTeam==1:
              population.append(dfTMP)
      return population

  def fitness_score(self,population):
      scores = []
      total_points=0
      for element in population:
          # Total Points Calculation
          total_points=element.total_points.sum()
          scores.append(total_points)
      scores = np.array(scores) 
      inds = (np.argsort(scores)[::-1])
      return scores, inds
  
  # tournament selection
  def selection2(pop, scores, k=3):
      # first random selection
      selection_ix = random.randint(0,len(pop))
      for ix in random.randint(0, len(pop), k-1):
          # check if better (e.g. perform a tournament)
          if scores[ix] < scores[selection_ix]:
              selection_ix = ix
      return pop[selection_ix]
    
  def selection(self,pop_after_fit, scores):
      population_nextgen = []
      # Keep best team
      population_nextgen.append(pop_after_fit[0])
      selectTeam=pop_after_fit[0]
      for x in range(1, len(pop_after_fit)):
        selection_ix = random.randint(0,len(pop_after_fit)-1)
        selectedTeam=pop_after_fit[x]
        xtmp=random.randint(0, len(pop_after_fit)-1)
        if xtmp>0:
          for k in range(0,xtmp):
            if scores [k] > scores[selection_ix]:
              selectTeam=pop_after_fit[k]
        population_nextgen.append(selectTeam)
      
      return population_nextgen
  
  def crossover(self,pop_after_sel):
      population_nextgen=[]
      population_nextgen.append(pop_after_sel[0])
      for i in range(1,len(pop_after_sel)):
            childTMP=pop_after_sel[i]
            GenesToChange=random.randint(0,int(len(childTMP)*self.Config["crossover_ratio"]))
            if GenesToChange > 0:
              GenesToKeep=len(childTMP)-GenesToChange
              validTeam=0
              k=0
              while validTeam ==0 and k < 150:
                possiblePartners=list(set(range(len(pop_after_sel)))-set([i]))
                SelectedPartner=(random.sample(possiblePartners, 1))[0]
                child=pd.concat([childTMP.sample(GenesToKeep), pop_after_sel[SelectedPartner].sample(GenesToChange)]).sort_values(by=['position', 'total_points'])
                validTeam=1
                # Check if the chromossome already exists
                validTeam=self.TeamValidation(child, population_nextgen)
                k+=1
                if validTeam==1:
                  population_nextgen.append(child)
            else:
                population_nextgen.append(childTMP)
      return population_nextgen

  
  def TeamValidation(self,chromosome, population_nextgen):
              validTeam=1
              # Check max number of players per position
              if len(chromosome[chromosome["position"]=="GK"])!=self.Config["SquadComposition"]["Goalkeepers"] or \
                 len(chromosome[chromosome["position"]=="DEF"])!=self.Config["SquadComposition"]["Defenders"] or \
                 len(chromosome[chromosome["position"]=="MID"])!=self.Config["SquadComposition"]["Midfielders"] or \
                 len(chromosome[chromosome["position"]=="FWD"])!=self.Config["SquadComposition"]["Forwards"]:
                  validTeam=0
              #Check Max elements per team
              if max(chromosome.groupby(['team']).size()) > self.Config["SquadComposition"]["Sameteam"]:
                  validTeam=0
              #Check max Cost
              if chromosome["value"].sum() > self.Config["SquadComposition"]["MaxCost"]:
                  validTeam=0
              #duplicayed player
              if max(chromosome.groupby(['element']).size()) > 1:
                      validTeam=0
              # Check if team already exists
              for dfRef in population_nextgen:
                if chromosome.equals(dfRef):
                    validTeam=0
                    break
              return validTeam      
            

  def mutation(self,pop_after_cross):
      population_nextgen = []
      population_nextgen.append(pop_after_cross[0])
      for i in range(1,len(pop_after_cross)):
          chromosome = pop_after_cross[i]
          validTeam=0
          k=0
          while validTeam ==0 and k < 150:
              x=chromosome.sample(int(len(chromosome)*self.Config["mutation_rate"]))
              chromosome2=chromosome[~chromosome.element.isin(list(x["element"]))]
              for  row in x[["position", "element"]].iterrows(): # checks a player of the same type but not in the current team
                y=self.dfPlayers[ (self.dfPlayers["position"] == row[1][0]) & (~self.dfPlayers.element.isin(x["element"])) ].sample(1)
                chromosome2=pd.concat([chromosome2, y])
              validTeam=self.TeamValidation(chromosome2, population_nextgen)
              # Check if the chromossome already exists
              k+=1
              if validTeam==1:
                population_nextgen.append(chromosome2)
      return population_nextgen
    
    
  def Execute(self, runid):
      best_chromo= []
      best_score= []
      scores_Dset=[]
      scorePre=0
      stopCriteria=0
      print("> Start Initial population...")
      population_nextgen=self.initilization_of_population()
      for i in range(self.Config["n_gen"]):
        scores,inds = self.fitness_score(population_nextgen)
        pop_after_fit=[]
        for x in inds:
          pop_after_fit.append(population_nextgen[x])
        pop_after_sel = self.selection(pop_after_fit, scores[inds])
        pop_after_cross = self.crossover(pop_after_sel)
        population_nextgen = self.mutation(pop_after_cross)
        best_chromo.append(pop_after_fit[inds[0]])
        best_score.append(scores[inds[0]])
        scores_Dset.append(scores[inds])
        if scores[inds[0]] > scorePre:
            scorePre=scores[inds[0]]
            stopCriteria=0
        else:
            stopCriteria+=1
        
        if stopCriteria==self.Config["stopCriteria"]:
          print(f"> Stopped for reaching stop Criteria value {stopCriteria} ...")
          break
        print("{2}: Pop:{4} Iteration>{0}/{3},  Best Score>{1}".format(i,scores[inds[0]], runid, self.Config["n_gen"], self.Config["PopSize"]))
      
      self.best_chromo=best_chromo
      self.best_score=best_score
      self.scores_Dset=scores_Dset
      return best_chromo, best_score, scores_Dset


  def StoreResults(self, DataFolder, runid):


    with open('{0}/mlflowLogs/GA_bestChromo_{1}.pickle'.format(DataFolder,runid ), 'wb') as handle:
      pickle.dump(self.best_chromo, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('{0}/mlflowLogs/GA_scores_{1}.pickle'.format(DataFolder, runid ), 'wb') as handle:
      pickle.dump(self.scores_Dset, handle, protocol=pickle.HIGHEST_PROTOCOL)



  def plotScores(self, DataFolder, runid):
    import numpy as np
    LDistinctChX=[]
    scores=[]
    #creates a list without duplication
    for k in self.best_chromo:
      found=0
      for l in LDistinctChX:
        if k.equals(l):
          found=1
          break
      if found==0:
        scores.append(k["total_points"].sum())
        LDistinctChX.append(k)
    inds = (np.argsort(scores)[::-1])

    LDistinctCh=[]
    ListScores=[]

    for x in inds:
      LDistinctCh.append(LDistinctChX[x])
    k=0
    for x in (self.best_score):
      k+=1
      ListScores.append({"generation":k, "score":x})
      
    plt.figure(figsize=(25,8))
    sns.set(style='dark',)
    sns.set_style("darkgrid")
    ax=sns.lineplot(data=pd.DataFrame(reversed(ListScores)), x="generation", y="score", markers=True, dashes=False)
    ax.set_title("Best Score Evolution")
    #plt.show()
    self.LDistinctCh=LDistinctCh
    plt.savefig('{0}/mlflowLogs/GA_scores_{1}.png'.format(DataFolder, runid ))

  def PlotSquads(self, DataFolder, runid):
    NumSolutions=1+self.Config["MaxSquadsToPick"]
    fig, axs = plt.subplots(nrows=NumSolutions,figsize=(40, 24))
    sns.set_style("darkgrid")

    for k in range(0,NumSolutions,1):
      sns.barplot( y = "total_points", x = "name", hue="position",  data = self.LDistinctCh[k].sort_values(by=['position', 'total_points']), ax=axs[k])
      fig.axes[k].set_title("Total Points {0}".format(self.LDistinctCh[k]["total_points"].sum()), fontsize = 20)
      dfTMP=self.LDistinctCh[k]
      dfTMP["solution"]=k
      if k==0:
        dfsolutions=dfTMP
      else:
        dfsolutions=dfsolutions.append(dfTMP)

    dfsolutions.to_csv('{0}/mlflowLogs/GA_Teams_{1}.csv'.format(DataFolder, runid), index=False, header=True)
    
    for ax in fig.axes:
        ax.set_xlabel("Player", fontsize = 20)
        ax.set_ylabel("Total Points", fontsize = 20)
        
        xlabels = [t.get_text()  for t in ax.get_xticklabels()]
        ax.set_xticklabels(xlabels,fontsize=19)
        ax.legend(loc='center right', title='Element Type')
        ax.tick_params(labelrotation=30)
        ax
    fig.tight_layout()
    plt.savefig('{0}/mlflowLogs/GA_Teams_{1}.png'.format(DataFolder, runid ))

    
  def TopPlayers(self,DataFolder, runid):

    # concatenate 
    TopRatio=0.2
    NTopTeams=int(len(self.LDistinctCh)*TopRatio)
    print("----> Top {0} of {1} teams".format(NTopTeams, len(self.LDistinctCh)))

    LDistinctChX=[]
    for x in range(0,NTopTeams,1):
      LDistinctChX.append(self.LDistinctCh[x])

    dfTopTeams = pd.concat(LDistinctChX)
    dfTopPlayers=pd.DataFrame((dfTopTeams.groupby(["element","name", "position"]).size()))
    dfTopPlayers.reset_index(inplace=True)
    dfTopPlayers.columns=["element", "name","position", "freq"]
    dfTopPlayers=dfTopPlayers.sort_values(by=['position', 'freq'], ascending=[True,False])
    dfTopPlayers["rank"]=dfTopPlayers.groupby("position")["freq"].rank("dense", ascending=False)
    dfTopPlayers[dfTopPlayers["rank"]<=3]
    dfTopPlayers.to_csv('{0}/mlflowLogs/GA_TopPlayers_{1}.csv'.format(DataFolder, runid ), header=True, index=False)


